# Character Animations
This is a bonus. Let's try to animate these characters.

In [1]:
import settings
from model import Story

# The last time json data was saved was step 4
story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

In [1]:
%pip install --upgrade transformers accelerate diffusers imageio-ffmpeg tbb torchvision torch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 58.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 57.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 59.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 60.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 60.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 60.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 59.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Generate list of scene images

Let's stick the characters into a scene so that we can have lots of context.

In [2]:
from PIL import Image
import settings

def place_character_in_scene(image: Image, background: Image=None):
    width = settings.CHARACTER_ANIMATION_WIDTH
    height = settings.CHARACTER_ANIMATION_HEIGHT

    # Create a transparent background
    resize_image = Image.new('RGBA', (width, height), (0, 0, 0, 0))

    if background:
        # Add background to the image
        scene = background.convert("RGBA")

        # Scale the scene to fit the normal dimensions while preserving aspect ratio
        scene_ratio = max(width / scene.width, height / scene.height)
        new_scene_size = (int(scene.width * scene_ratio), int(scene.height * scene_ratio))
        scene = scene.resize(new_scene_size)
        
        # Calculate the position to paste the scene image onto the background
        scene_offset = ((width - new_scene_size[0]) // 2, (height - new_scene_size[1]))
        
        # Paste the scene image onto the background
        resize_image.paste(scene, scene_offset, scene)


    # Calculate the position to paste the source image onto the background
    src_width, src_height = image.size
    offset = ((width - src_width) // 2, (height - src_height))
    # offset = ((width - src_width) // 2, (height - src_height) // 2)

    # Ensure the image has an alpha channel
    image = image.convert("RGBA")
    
    # Paste the source image onto the background
    resize_image.paste(image, offset, image)

    # Convert the image to RGB
    resize_image = resize_image.convert("RGB")

    return resize_image

In [3]:
from IPython.display import display, Markdown
import rembg
from diffusers.utils import export_to_gif
import os
import random
from utils import deindent

for character in story.characters:
    dst_image_path = f"{settings.STORY_DIR}/step_7/characters/{character.nickname}_scene.png"

    # Skip it if we already made it
    if not os.path.exists(dst_image_path):
        act = random.choice(story.acts)
        scene = random.choice(act.scenes)

        src_image_path = f"{settings.STORY_DIR}/step_5/characters/{character.nickname}.png"
        src_image = Image.open(src_image_path)

        # Make the character a bit smaller
        src_image = src_image.resize((src_image.size[0]//3*2, src_image.size[1]//3*2))

        # Remove the background
        src_image = src_image.convert("RGBA")
        src_image = rembg.remove(src_image)

        background = Image.open(f"stories/my_story/step_6/scenes/{scene.scene_id}.png")

        scene_image = place_character_in_scene(src_image, background=background)

        dir_name = os.path.dirname(dst_image_path)
        os.makedirs(dir_name, exist_ok=True)

        scene_image.save(dst_image_path)

    display(Markdown(deindent(f"""
        ---
        ## {character.name}

        ![Animation of {character.name}]({dst_image_path})

    """)))

---
## Meera 'Midnight' Singh

![Animation of Meera 'Midnight' Singh](stories/my_story/step_7/characters/meera_scene.png)

---
## Jax 'Specter' Lee

![Animation of Jax 'Specter' Lee](stories/my_story/step_7/characters/jax_scene.png)

---
## Dr. Zhang 'Zen' Wei

![Animation of Dr. Zhang 'Zen' Wei](stories/my_story/step_7/characters/dr_zhang_scene.png)

---
## Maya 'Rampart' Patel

![Animation of Maya 'Rampart' Patel](stories/my_story/step_7/characters/maya_scene.png)

---
## The Architect (Erebus)

![Animation of The Architect (Erebus)](stories/my_story/step_7/characters/architect_scene.png)

---
## The Nexus Guardian (Astrum)

![Animation of The Nexus Guardian (Astrum)](stories/my_story/step_7/characters/nexus_guardian_scene.png)

---
## Kaito 'Sparrow' Hernandez

![Animation of Kaito 'Sparrow' Hernandez](stories/my_story/step_7/characters/earth_rebel_scene.png)

---
## Regina 'The Shark' Thornton

![Animation of Regina 'The Shark' Thornton](stories/my_story/step_7/characters/corporate_exec_scene.png)

# Generate list of animation descriptions
We will use vision to look at the character and scene. Then generate a description of the animation from that.

# Animate Each Character


In [4]:
from model import Story
import settings

story = Story.load_from_directory(settings.STORY_DIR + "/step_4")

### Image/Text-to-video

#### CogVideoX-5b-I2V
https://huggingface.co/THUDM/CogVideoX-5b-I2V

#### i2vgen-xl
https://huggingface.co/docs/diffusers/main/en/using-diffusers/text-img2vid#i2vgen-xl


In [5]:
from IPython.display import display, Markdown
from model_image2video import image_to_video
from utils import deindent
from PIL import Image

for c in story.characters:
    src_image_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}_scene.png"
    src_image = Image.open(src_image_path)
    
    prompt = f"slow camera full shot arc around focusing on a {c.gender} {c.race} {c.role} age {c.age}. Hair flowing naturally. {c.physical_appearance}"
    # prompt = f"{c.animation_description}. {scene.background_animation}"
    # prompt = f"An slow epic arc shot to the right while zooming in. Grenades are exploding all around the ninja attacks and dust is flying everywhere. {scene.background_animation}"
    # prompt = f"a {c.gender} {c.race} speaks animatedly, gesturing with their hands. Lasers shoot out of their eyes and an explosion ensues."

    formatted_prompt = "\n".join([f"> {line}" for line in prompt.split("\n")])
    dst_image_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}.gif"
    dst_image_small_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}_small.gif"
    dst_video_path = f"{settings.STORY_DIR}/step_7/characters/{c.nickname}.mp4"
    guidance_scale = 7.0
    
    # Don't re-render the video if it already exists
    # if not os.path.exists(dst_image_small_path):
    image_to_video(prompt=prompt, image=src_image, 
                    video_filename=dst_video_path, 
                    gif_filename=dst_image_path,
                    gif_small_filename=dst_image_small_path,
                    display_video=False, loop_reverse=True, sequences=1)

    display(Markdown(deindent(f"""
        ---
                
        ## {c.name}

        **Role**: {c.role}
                            
        **Physical Apperaance**: {c.physical_appearance}

        **Description**: {c.description}

        **Personality**: {c.personality}

        ### Prompt:

        **guidance_scale**: {guidance_scale}

        {formatted_prompt}

        ![Animation of {c.name}]({dst_image_path})

    """)))

    display(Markdown(f"Full size GIF: [{dst_image_path}](./{dst_image_path})"))
    display(Markdown(f"Full size MP4: [{dst_video_path}](./{dst_video_path})"))

RuntimeError: Failed to import diffusers.pipelines.cogvideo.pipeline_cogvideox_image2video because of the following error (look up to see its traceback):
Failed to import transformers.models.t5.modeling_t5 because of the following error (look up to see its traceback):
operator torchvision::nms does not exist